In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd
from torchvision import transforms, models
from torch.utils.data import DataLoader
from PIL import Image
from sklearn.model_selection import train_test_split

In [3]:
import os
import csv

# Set the directory 
root_dir = 'Dataset\Dataset\Training_data'  
output_csv = 'animal_images.csv' 

# List to hold all (image path, label) tuples
image_paths = []

# Walk through the directory and gather data
for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  
            print(root)
            label = os.path.basename(root)  # Folder name as label
            path = os.path.join(root, file)  # Full path to the image
            image_paths.append((path, label))

# Write data to CSV
with open(output_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'label'])  # Header row
    for path, label in image_paths:
        writer.writerow([path, label])




Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
Dataset\Dataset\Training_data\badger
D

In [4]:
class AnimalDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        # Convert label to torch.long
        label = torch.tensor(label, dtype=torch.long)

        return image, label


In [5]:
# Parameters
num_classes = 6
batch_size = 32
learning_rate = 0.001
num_epochs = 10
image_size = 224
csv_file = 'animal_images.csv'  # Path to your CSV file

# Data transformations
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

# Read CSV and prepare dataset
df = pd.read_csv('animal_images.csv')

# Convert labels to numerical values and ensure they are integers
label_mapping = {label: idx for idx, label in enumerate(df['label'].unique())}
df['label'] = df['label'].map(label_mapping)
# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2)

# Create dataset objects
train_dataset = AnimalDataset(dataframe=train_df, transform=transform)
val_dataset = AnimalDataset(dataframe=val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [6]:
# Load pre-trained ResNet model and modify it
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

C:\Users\user1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
print(torch.cuda.is_available())

False


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

import matplotlib.pyplot as plt

# Variables for precision and recall calculation
class_TP = list(0. for i in range(num_classes))  # True Positives
class_FP = list(0. for i in range(num_classes))  # False Positives
class_FN = list(0. for i in range(num_classes))  # False Negatives

# Variables to store metrics per epoch for plotting
epoch_precision = {i: [] for i in range(num_classes)}
epoch_recall = {i: [] for i in range(num_classes)}
epoch_accuracy = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Reset metrics for each epoch
    class_TP = [0] * num_classes
    class_FP = [0] * num_classes
    class_FN = [0] * num_classes
    total, correct = 0, 0

    # Validation with precision and recall calculation
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            for i in range(labels.size(0)):
                label = labels[i]
                pred_label = predicted[i]
                if label == pred_label:
                    class_TP[label] += 1
                else:
                    class_FP[pred_label] += 1
                    class_FN[label] += 1

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate and store metrics per class
    for i in range(num_classes):
        precision = 100 * class_TP[i] / (class_TP[i] + class_FP[i]) if (class_TP[i] + class_FP[i]) > 0 else 0
        recall = 100 * class_TP[i] / (class_TP[i] + class_FN[i]) if (class_TP[i] + class_FN[i]) > 0 else 0
        epoch_precision[i].append(precision)
        epoch_recall[i].append(recall)

    # Store overall accuracy
    accuracy = 100 * correct / total
    epoch_accuracy.append(accuracy)
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy}%')

In [ ]:
# Plotting the metrics evolution
for i in range(num_classes):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 3, 1)
    plt.plot(epoch_precision[i], label=f'Class {i} Precision')
    plt.xlabel('Epoch')
    plt.ylabel('Precision')
    plt.title(f'Class {i} Precision Over Epochs')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(epoch_recall[i], label=f'Class {i} Recall')
    plt.xlabel('Epoch')
    plt.ylabel('Recall')
    plt.title(f'Class {i} Recall Over Epochs')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(epoch_accuracy, label='Total Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Total Accuracy Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Save the model
torch.save(model.state_dict(), 'animal_classifier_model.pth')